## 데이터셋 준비 하기
Instruction Fine Tuning을 위해 데이터셋을 준비합니다.  
데이터셋은 아래의 순서로 진행하려 합니다.
---
Instruction Tuning의 데이터셋은 어떻게 만들어야할까요?  
학습에 필요한 Instruction Dataset은 아래와 같은 데이터 형태입니다.
([Stanford Alpca 데이터셋](https://github.com/tatsu-lab/stanford_alpaca/blob/main/alpaca_data.json)을 참조했습니다)
```plain
질문 :
건강하기 위한 3가지 팁 알려줘.
답변 :
1. 충분한 양의 채소와 과일을 섭취해서 균형있는 식단을 가지세요.
2. 신체가 활동적이고 건강할 수 있도록 규칙적으로 운동을 하세요.
3. 충분한 수면시간을 가지고 규칙적인 수면 습관을 가지세요.
```
위와 같은 데이터셋을 만들려면 여러 방법이 있습니다. 개인이 데이터셋을 구축하기 쉬운 방법중 하나는 강력한 언어모델의 힘을 빌리는 것일 겁니다.  
이 예제에서는 ChatGPT를 활용하여 학습 데이터를 생성하였습니다.  
⭐ 24년 5월 14일 OpenAI에서 `gpt-4o`를 출시하여 더욱 저렴한 가격으로 사용할 수 있게 되었습니다.  

데이터셋은 1) 키워드 기반으로 질문을 생성, 2) 부동산 스터디 카페글을 크롤링하는 2가지 방법으로 생성해봤습니다.  

이 예제에서 데이터셋을 만드는 순서는 간단하게 아래와 같습니다.  

#### <span style="color: #F2D388;"> 1. 도메인 정하기  </span>
어떤 Assistant(챗봇)을 만들까?를 먼저 정해봅니다. 사내 봇? 주식 답변? 등등...  
이 예제에서는 부동산을 타겟으로 하였습니다.  


#### <span style="color: #F2D388;"> 2. 키워드 기반으로 데이터셋 만들기 </span>
데이터셋은 2가지 방법으로 생성해봤습니다.  
##### <span style="color: #ECA75D;"> 2-1. 부동산에 관련된 키워드 모으기 </span>
책, 뉴스, 부동산 사이트에서 키워드를 수집해보았습니다.  
사람들이 궁금해할만한 키워드를 대상으로 합니다.  
(예: 전세 계약, 신혼부부 특별공급, 토지거래허가구역...)  
키워드에 대한 예제 데이터는 [data/seed_words.txt](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/seed_words.txt)를 참조해주세요.  
##### <span style="color: #ECA75D;"> 2-2. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트를 만들기 </span>
2번에서 수집한 키워드를 기반으로, 사람들이 궁금해할만한 질문리스트를 ChatGPT를 활용해 만들어봅니다.  
(예: 전세계약 -> 전세계약할 때 주의해야할 점이 뭐가 있을까요?)  
검색 질문리스트 예제 데이터는 [data/questions_search.jsonl](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/questions_search.jsonl)를 참조해주세요. 


#### <span style="color: #F2D388;"> 3. 카페글 기반으로 데이터셋 만들기 </span>
##### <span style="color: #ECA75D;"> 3-1. 네이버 카페 글의 질문으로 질문 데이터셋 만들기 </span>
부동산 관련 카페중 가장 큰 규모인 부동산스터디 카페의 회원간 묻고 답하기 글을 크롤링합니다.  
##### <span style="color: #ECA75D;"> 3-2. 수집한 카페 글을 유사도 기준으로 필터링하기 </span>
카페글 중엔 부동산과 관련이 없는 데이터셋이 있을 수 있습니다. 따라서 그러한 데이터를 필터링합니다.  
수집된 데이터를 그대로 ChatGPT로 필터링하면 요금이 비싸기 때문에 먼저 인코더 모델로 유사도를 계산합니다.
##### <span style="color: #ECA75D;"> 3-3. 유사도로 필터링한 데이터를 ChatGPT로 한번 더 필터링하기 </span>
3-2번에서 필터링한 데이터를 ChatGPT로 한번 더 필터링합니다. 이 때 유사도 기반으로 필터링하지 못한 데이터들을 ChatGPT가 필터링해줄 수 있습니다.  

    
#### <span style="color: #F2D388;"> 4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기 </span>
2, 3번에서 수집한 질문리스트를 Selenium 라이브러리를 활용하여 네이버에 검색합니다. 검색 결과중 인기글의 텍스트 데이터를 추출합니다.  
인기글 문서 예제 데이터는 [data/documents.jsonl](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/documents.jsonl)를 참조해주세요.  


#### <span style="color: #F2D388;"> 5. Instuction 데이터셋 만들기 </span>
질문리스트 + 정렬한 인기글을 합쳐서 ChatGPT에 Instruction 데이터를 만들어달라고 요청합니다. 이 Instruction 데이터는 Fine Tuning에 사용됩니다.
인기글 문서 예제 데이터는 [data/instructions.jsonl](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/instructions.jsonl)를 참조해주세요.  

In [1]:
!pip install --quiet\
selenium==4.20.0\
openai==1.23.6\
python-dotenv==1.0.1


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## <span style="color: #F2D388;"> 2. 키워드 기반으로 데이터셋 만들기 </span>

### <span style="color: #ECA75D;"> 2-1. 부동산에 관련된 키워드 모으기 </span>
부동산에 관련된 키워드를 수집하여 저장합니다.  
예제 데이터는 [data/seed_words.txt](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/seed_words.txt)를 참조해주세요.

In [24]:
with open("./data/seed_words.txt", "r") as f:
    seed_words = f.readlines()

In [3]:
seed_words[:4]

['전세 계약\n', '임대차 계약\n', '전세 사기\n', '임대차 분쟁\n']

불필요한 newline이 있어서 제거해줍니다.

In [4]:
# Element마다 있는 \n 제거, '전세 계약\n' -> '전세 계약'
seed_words = list(map(lambda x: x.strip("\n"), seed_words))

In [5]:
seed_words[:4]

['전세 계약', '임대차 계약', '전세 사기', '임대차 분쟁']

### <span style="color: #ECA75D;"> 2-2. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트를 만들기 </span>
미리 작성된 프롬프트에 `seed_words`를 `format` 적용하여 프롬프트를 완성시킵니다.  
프롬프트 작성은 [chatgpt-prompt-engineering-for-developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers)와 [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txt)를 기반으로 작성하였습니다.

In [6]:
from prompts import SEED_WORD_PROMPT_PREFIX, SEED_WORD_PROMPT_CONTENT

# prompts.py에 프롬프트들 템플릿이 있습니다.
print(SEED_WORD_PROMPT_PREFIX + SEED_WORD_PROMPT_CONTENT)

당신은 부동산에 관심이 많은 사람입니다. 당신의 역할에 따라 주어진 seed_word에 대해 궁금해할 질문을 생성하세요.
seed_word는 총 5개씩 주어집니다.
seed_word에 대해 각각 20개의 질문을 생성해야합니다.
생성할 질문에 대한 요구 사항은 다음과 같습니다:
1. 지시대명사를 사용해서는 안됩니다. seed_word에 있는 명사를 그대로 사용하세요.
2. 이미 만들어낸 질문과 동일하거나 유사한 질문을 만들어내서는 안됩니다.
3. 만들어내는 질문들은 어휘의 다양성을 위해 다양한 단어를 사용해야 합니다.
4. 만들어내는 질문들은 문장의 다양성을 위해 의문문과 평서문을 모두 사용해야 합니다.
5. 반드시 한글로 질문을 만드세요.
6. 만들어낸 질문은 JSON형식을 따라야 하고, indent는 없어야 합니다.
7. 응답하는 답변 문자에는 줄바꿈, \n, \t, \b 등의 특수 문자가 없어야합니다.
8. seed_word에 대해 중복으로 질문을 생성했는지 확인하세요. seed_word에 대한 질문을 이미 생성했다면, 동일한 seed_word에 대한 작업을 해서는 안됩니다.
9. 아래 양식으로 출력하세요:
{{"seed_word": "주어진 seed_word", "answer": ["1번째 질문", "2번째 질문"... , "20번째 질문"]}}

seed_word는 다음과 같습니다:
{seed_word}


In [7]:
print(SEED_WORD_PROMPT_PREFIX + SEED_WORD_PROMPT_CONTENT.format(seed_word=seed_words[0]))

당신은 부동산에 관심이 많은 사람입니다. 당신의 역할에 따라 주어진 seed_word에 대해 궁금해할 질문을 생성하세요.
seed_word는 총 5개씩 주어집니다.
seed_word에 대해 각각 20개의 질문을 생성해야합니다.
생성할 질문에 대한 요구 사항은 다음과 같습니다:
1. 지시대명사를 사용해서는 안됩니다. seed_word에 있는 명사를 그대로 사용하세요.
2. 이미 만들어낸 질문과 동일하거나 유사한 질문을 만들어내서는 안됩니다.
3. 만들어내는 질문들은 어휘의 다양성을 위해 다양한 단어를 사용해야 합니다.
4. 만들어내는 질문들은 문장의 다양성을 위해 의문문과 평서문을 모두 사용해야 합니다.
5. 반드시 한글로 질문을 만드세요.
6. 만들어낸 질문은 JSON형식을 따라야 하고, indent는 없어야 합니다.
7. 응답하는 답변 문자에는 줄바꿈, \n, \t, \b 등의 특수 문자가 없어야합니다.
8. seed_word에 대해 중복으로 질문을 생성했는지 확인하세요. seed_word에 대한 질문을 이미 생성했다면, 동일한 seed_word에 대한 작업을 해서는 안됩니다.
9. 아래 양식으로 출력하세요:
{{"seed_word": "주어진 seed_word", "answer": ["1번째 질문", "2번째 질문"... , "20번째 질문"]}}

seed_word는 다음과 같습니다:
전세 계약


GPT모델의 토큰 수 계산은 Input도 포함됩니다. 따라서 불필요하게 `SEED_WORD_PROMPT_PREFIX`를 모든 키워드에 반복해서 전달할 필요가 없습니다.  

In [8]:
# 모델에게 Prefix 토큰을 중복해서 전달할 필요가 없으므로, Prefix 토큰은 1번만 사용하고 Content을 반복해서 프롬프트를 생성
seed_word_prefix = SEED_WORD_PROMPT_PREFIX
seed_word_content = [SEED_WORD_PROMPT_CONTENT.format(seed_word=s) for s in seed_words]

In [9]:
# 1개의 Prefix 마다 n_content개의 Content를 추가합니다.
n_content = 5

seed_word_prompts = []
total_prompts = len(seed_words) // n_content
for idx in range(1, total_prompts+2):
    start_index = (idx -1) * n_content
    end_index = idx * n_content
    seed_word_prompt = seed_word_prefix + "\n".join(seed_word_content[start_index:end_index])
    seed_word_prompts.append(seed_word_prompt)

In [10]:
print(seed_word_prompts[0])

당신은 부동산에 관심이 많은 사람입니다. 당신의 역할에 따라 주어진 seed_word에 대해 궁금해할 질문을 생성하세요.
seed_word는 총 5개씩 주어집니다.
seed_word에 대해 각각 20개의 질문을 생성해야합니다.
생성할 질문에 대한 요구 사항은 다음과 같습니다:
1. 지시대명사를 사용해서는 안됩니다. seed_word에 있는 명사를 그대로 사용하세요.
2. 이미 만들어낸 질문과 동일하거나 유사한 질문을 만들어내서는 안됩니다.
3. 만들어내는 질문들은 어휘의 다양성을 위해 다양한 단어를 사용해야 합니다.
4. 만들어내는 질문들은 문장의 다양성을 위해 의문문과 평서문을 모두 사용해야 합니다.
5. 반드시 한글로 질문을 만드세요.
6. 만들어낸 질문은 JSON형식을 따라야 하고, indent는 없어야 합니다.
7. 응답하는 답변 문자에는 줄바꿈, \n, \t, \b 등의 특수 문자가 없어야합니다.
8. seed_word에 대해 중복으로 질문을 생성했는지 확인하세요. seed_word에 대한 질문을 이미 생성했다면, 동일한 seed_word에 대한 작업을 해서는 안됩니다.
9. 아래 양식으로 출력하세요:
{{"seed_word": "주어진 seed_word", "answer": ["1번째 질문", "2번째 질문"... , "20번째 질문"]}}

seed_word는 다음과 같습니다:
전세 계약
임대차 계약
전세 사기
임대차 분쟁
깡통 전세


이제 ChatGPT에게 데이터를 작성하도록 합니다.  
💡 ChatGPT API를 사용하기 전에, [ChatGPT 모델 가격 정책](https://openai.com/pricing)을 참조하세요.  
ChatGPT는 지속적으로 업데이트를 하는데, 과거버전은 사용하지 않도록 합니다.  
과거버젼은 퍼포먼스도 안좋은데 가격이 비싸므로 사용할 이유가 없습니다.  
여기서는 `gpt-4-turbo`를 사용합니다.

데이터 전처리는 spark, hive 등 분산 처리 시스템 / 멀티 쓰레딩, 멀피 프로세싱 / Concurrency와 같은 작업을 활용하면 속도가 많이 개선됩니다.  
(파이썬이 GIL로 인해 진정한 멀티쓰레딩이 불가능하더라도, I/O 작업이 수반된다면 Concurrency를 통해 효율성이 개선됩니다.)  
이 예제의 ChatGPT API는 [Multi Threading](https://docs.python.org/ko/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor)를 사용합니다.

In [13]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Worker의 수는 32개 또는 코어수 +4개가 디폴트값입니다. - 공식 가이드 문서 참조
with ThreadPoolExecutor() as pool:
    questions = list(tqdm(pool.map(partial(get_completion, model="gpt-4-turbo"), seed_word_prompts)))
    
with open("./data/questions_keyword.jsonl", "a") as f:
    f.write("\n".join(questions))

55it [14:13, 15.53s/it]


In [17]:
from utils import jload

jload("./data/questions_keyword.jsonl")[0]

{'seed_word': '전세 계약',
 'answer': ['전세 계약을 체결할 때 필요한 서류는 무엇인가요?',
  '전세 계약 시 주의해야 할 점은 무엇인가요?',
  '전세 계약 갱신 청구권은 어떻게 적용되나요?',
  '전세 계약을 해지하려면 어떤 절차를 밟아야 하나요?',
  '전세 계약 시 보증금 반환 문제는 어떻게 해결하나요?',
  '전세 계약 기간 중 집주인이 바뀌면 어떻게 되나요?',
  '전세 계약을 할 때 중개수수료는 얼마인가요?',
  '전세 계약 시 보증금을 보호받을 수 있는 방법은 무엇인가요?',
  '전세 계약 갱신 시 임대료 인상률은 어떻게 결정되나요?',
  '전세 계약 체결 전에 확인해야 할 주택 상태는 무엇인가요?',
  '전세 계약 시 임차인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 임대인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 확정일자는 왜 중요한가요?',
  '전세 계약 시 전입신고는 언제 해야 하나요?',
  '전세 계약 갱신 거절 시 임차인은 어떤 조치를 취할 수 있나요?',
  '전세 계약 시 보증금을 전액 돌려받지 못하는 경우는 어떤 경우인가요?',
  '전세 계약 시 임대인이 파산하면 보증금은 어떻게 되나요?',
  '전세 계약 시 임대인과 임차인 간의 분쟁 해결 방법은 무엇인가요?',
  '전세 계약 시 임대인이 계약을 일방적으로 해지할 수 있나요?',
  '전세 계약 시 임차인이 보증금을 늦게 받는 경우 어떻게 대처해야 하나요?']}

## <span style="color: #F2D388;"> 3. 카페글 기반으로 데이터셋 만들기 </span>

### <span style="color: #ECA75D;"> 3-1. 네이버 카페 글의 질문으로 질문 데이터셋 만들기 </span>
부동산 관련 카페중 가장 큰 규모인 부동산스터디 카페의 회원간 묻고 답하기 글을 크롤링합니다.  
(`get_document_through_selenium` 함수는 내부적으로 Multiprocessing으로 구성되어있습니다)

In [3]:
from utils import get_document_through_selenium

cafe_data = get_document_through_selenium(
    crawling_type="cafe",
    n_page=200,
    indent=4,
    mode="w",
    save_path = "./data/questions_cafe.json"
)

200it [01:02,  3.20it/s]


In [3]:
from utils import jload

jload("./data/questions_cafe.json")[0]['document'][:10]

['[LG전자 베스트샵 봉천센트럴점]🥳RENEWAL OPEN 행사🥳 (광고)',
 '[LG전자 베스트샵 롯데 수원점] 리뉴얼 GRAND OPEN 가전제품 세일 💕',
 '부동산 스터디 카페에 올라오는 주식 홍보방들은 모두 사기글입니다.',
 '[공지]붇스터디게시판 운영가이드(2023.9.3 수정추가)',
 '부동산 스터디 카페 내에 계정 해킹 사례가 너무 많네요',
 '전라도 사람들은 투표권 주지말자',
 '고수님들은 3억 자금이 있다면 무엇을 하시나요?',
 '용산 vs 강남 신혼부부 집 조언부탁드립니다!',
 '서대문구, 종로구에서 갭 3억~3.5억 투자 추천아파트',
 '고소득 맞벌이 부부 매수 고민']

### <span style="color: #ECA75D;"> 3-2. ChatGPT로 필터링하기 </span>
수집한 데이터에서 부동산에 관련이 있는 데이터들만 필터링합니다.  
ChatGPT에 요청하여 이러한 데이터들을 필터링할 수 있게 합니다.  
💡 충분히 파인튜닝된 인코더 모델이 있다면, Similarity로 분류한 뒤 ChatGPT로 분류하면 토큰 비용을 아낄 수 있습니다.

In [1]:
from utils import jload

questions_cafe = jload("./data/questions_cafe.json")

In [2]:
questions_cafe[0]['document'][:3]

['강동역SK리더스뷰 파격조건 선착순 동호지정계약 진행중 (광고',
 '[LG전자 베스트샵 봉천센트럴점]🥳RENEWAL OPEN 행사🥳 (광고)',
 '부동산 스터디 카페에 올라오는 주식 홍보방들은 모두 사기글입니다.']

`List[Dict[List]]`의 중첩된 형태이므로, flatten하여 list 형태로 바꿔줍니다.

In [3]:
from itertools import chain

documents = [q['document'] for q in questions_cafe]
questions_cafe = list(chain(*documents))

약 10000개의 게시글 타이틀이 수집되어 있습니다.

In [4]:
len(questions_cafe)

10008

이번에는 gpt-3.5-turbo를 사용해보겠습니다.  
gpt-4-turbo보다 성능이 떨어지기 때문에 프롬프트에 few-shot을 넣어주겠습니다.  

In [5]:
from prompts import CAFE_PROMPT_PREFIX, CAFE_PROMPT_CONTENT

In [6]:
print(CAFE_PROMPT_PREFIX + CAFE_PROMPT_CONTENT)

당신은 부동산에 관심이 많은 사람입니다. documents를 요구사항에 따라 필터링하고 질문을 생성해야합니다.
documents는 20개가 주어지며, 필터링한 documents를 수정하여 명확하고 구어적인 표현으로 질문을 생성해야 합니다.
요구사항은 다음과 같습니다:
1. 정치적 발언, 혐오 발언 등 독성있고 해로운 documents는 제외하세요.
2. 부동산 분야와 관련된 documents만 필터링하세요. 조금도 부동산에 관련이 없다면 과감히 제외하세요.
3. 필터링한 documents로 사람들이 궁금해 할만한 질문을 생성하세요.
4. 필터링한 documents는 출력 결과에 포함하지마세요.
5. 출력한 데이터는 JSON형식을 따라야하고, indent는 없어야 합니다.
6. 아래 양식으로 출력하세요.
"생성한 질문1"
"생성한 질문2"
...

예시1:
documents = '청약시 무주택자격 질문'
생성한 질문 = '청약시 무주택 자격이 어떻게 되나요?'

예시2:
documents = '여의도 매수하려고 하는데요.'
생성한 질문 = '여의도 매수 관하여 질문드립니다.'

예시3:
documents = '서울 아파트 매물이 빠른 속도로 소진되고 있네요...'
생성한 질문 = '서울 아파트 매물이 추세가 어떻게 되고 있나요?'

예시4:
documents = '매매 전세 동시진행'
생성한 질문 = '매매와 전세를 동시에 진행하는데 주의할 점이 뭘까요?'

예시5:
documents = '3기신도시 당해 아닌경우'
생성한 질문 = '3기신도시 당해 조건에 해당하지 않는 경우 문의드립니다.'

###documents:
{documents}



1개의 프롬프트에 20개의 documents를 추가해줍니다.  

In [7]:
n_queries = 20

queries = []
batch = len(questions_cafe) // n_queries
for idx in range(1, batch+2):
    start_index = (idx -1) * n_queries
    end_index = idx * n_queries
    query_documents = questions_cafe[start_index:end_index]
    query_documents = "\n".join(query_documents)
    queries.append(CAFE_PROMPT_PREFIX + CAFE_PROMPT_CONTENT.format(documents=query_documents))

In [8]:
print(queries[0])

당신은 부동산에 관심이 많은 사람입니다. documents를 요구사항에 따라 필터링하고 질문을 생성해야합니다.
documents는 20개가 주어지며, 필터링한 documents를 수정하여 명확하고 구어적인 표현으로 질문을 생성해야 합니다.
요구사항은 다음과 같습니다:
1. 정치적 발언, 혐오 발언 등 독성있고 해로운 documents는 제외하세요.
2. 부동산 분야와 관련된 documents만 필터링하세요. 조금도 부동산에 관련이 없다면 과감히 제외하세요.
3. 필터링한 documents로 사람들이 궁금해 할만한 질문을 생성하세요.
4. 필터링한 documents는 출력 결과에 포함하지마세요.
5. 출력한 데이터는 JSON형식을 따라야하고, indent는 없어야 합니다.
6. 아래 양식으로 출력하세요.
"생성한 질문1"
"생성한 질문2"
...

예시1:
documents = '청약시 무주택자격 질문'
생성한 질문 = '청약시 무주택 자격이 어떻게 되나요?'

예시2:
documents = '여의도 매수하려고 하는데요.'
생성한 질문 = '여의도 매수 관하여 질문드립니다.'

예시3:
documents = '서울 아파트 매물이 빠른 속도로 소진되고 있네요...'
생성한 질문 = '서울 아파트 매물이 추세가 어떻게 되고 있나요?'

예시4:
documents = '매매 전세 동시진행'
생성한 질문 = '매매와 전세를 동시에 진행하는데 주의할 점이 뭘까요?'

예시5:
documents = '3기신도시 당해 아닌경우'
생성한 질문 = '3기신도시 당해 조건에 해당하지 않는 경우 문의드립니다.'

###documents:
강동역SK리더스뷰 파격조건 선착순 동호지정계약 진행중 (광고
[LG전자 베스트샵 봉천센트럴점]🥳RENEWAL OPEN 행사🥳 (광고)
부동산 스터디 카페에 올라오는 주식 홍보방들은 모두 사기글입니다.
[공지]붇스터디게시판 운영가이드(2023.9.3 수정추가)
부동산 스터디 카페 내에 계정 해킹 사례가 너무 많네요
전라도 사람들은 투표권 주지말자
5

프롬프트를 작성할 때, gpt-3.5-turbo는 아래와 같이 프롬프트를 작성하는 것이 도움이 되었습니다.  
+ Dictionary나 List 형태보다 텍스트 + 줄바꿈 규칙같은 단순한 규칙을 잘 지킴
+ Instruction을 위배할 때가 많기 때문에 소량의 데이터로 많이 테스트 해봐야 함

gpt-3.5-turbo는 저렴하지만 데이터 생성에 어려움이 클 정도로 gpt-4보다 확실히 성능이 많이 떨어집니다.(수작업이 많이 필요하네요.)  

In [14]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from tqdm import tqdm
from utils import get_completion
import os

with ThreadPoolExecutor(os.cpu_count()) as pool:
    result = list(tqdm(pool.map(partial(get_completion, model="gpt-3.5-turbo"), queries)))
    with open("./data/questions_cafe_filtered.txt", "a", encoding="utf-8") as f:
        for r in result:
            for line in r.split("\n"):
                f.write(line)
                f.write("\n")

501it [05:04,  1.64it/s]


크롤링한 데이터에서 정치적 발언, 혐오 발언이 아직 포함되어 있어서 직접 데이터를 체크하여 제거해주었습니다.  

In [21]:
with open("./data/questions_cafe_filtered.txt", "r")as f:
    questions_cafe_filtered = f.readlines()

In [28]:
from utils import jsave
jsave(res_list, "./data/questions_cafe_filtered.json", indent=4)

gpt-3.5-turbo가 제대로 필터링하거나 규칙에 따른 양식을 지키지 못하는 경우가 많습니다.  
예제를 작성하면서 직접 데이터를 체크하였습니다.  

In [33]:
jload("./data/questions_cafe_filtered.json")[:4]

['신혼부부 집 선택 시 고려해야 할 사항은 무엇인가요?',
 '고소득 맞벌이 부부가 고민하는 매수 시 어떤 점을 고려해야 하나요?',
 '강남권 아파트 중에서 골프장이 잘 되어 있는 곳은 어디인가요?',
 '40평대 아파트의 입지 조건에 대해 알려주세요.']

## <span style="color: #F2D388;"> 4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기 </span>
2번, 3번에서 생성한 질문리스트를 selenium 라이브러리를 통해 네이버로 검색합니다.  
검색 결과의 인기글의 텍스트 정보를 저장합니다.  

In [1]:
from utils import jload

questions_keyword = jload("./data/questions_keyword.jsonl")

In [2]:
questions_keyword[0]

{'seed_word': '전세 계약',
 'answer': ['전세 계약을 체결할 때 필요한 서류는 무엇인가요?',
  '전세 계약 시 주의해야 할 점은 무엇인가요?',
  '전세 계약 갱신 청구권은 어떻게 적용되나요?',
  '전세 계약을 해지하려면 어떤 절차를 밟아야 하나요?',
  '전세 계약 시 보증금 반환 문제는 어떻게 해결하나요?',
  '전세 계약 기간 중 집주인이 바뀌면 어떻게 되나요?',
  '전세 계약을 할 때 중개수수료는 얼마인가요?',
  '전세 계약 시 보증금을 보호받을 수 있는 방법은 무엇인가요?',
  '전세 계약 갱신 시 임대료 인상률은 어떻게 결정되나요?',
  '전세 계약 체결 전에 확인해야 할 주택 상태는 무엇인가요?',
  '전세 계약 시 임차인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 임대인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 확정일자는 왜 중요한가요?',
  '전세 계약 시 전입신고는 언제 해야 하나요?',
  '전세 계약 갱신 거절 시 임차인은 어떤 조치를 취할 수 있나요?',
  '전세 계약 시 보증금을 전액 돌려받지 못하는 경우는 어떤 경우인가요?',
  '전세 계약 시 임대인이 파산하면 보증금은 어떻게 되나요?',
  '전세 계약 시 임대인과 임차인 간의 분쟁 해결 방법은 무엇인가요?',
  '전세 계약 시 임대인이 계약을 일방적으로 해지할 수 있나요?',
  '전세 계약 시 임차인이 보증금을 늦게 받는 경우 어떻게 대처해야 하나요?']}

In [3]:
# 키워드로 생성한 데이터
from itertools import chain

# answer를 추출해서 [[query1], [query2]] 구조를 [query1, query2]로 flatten해주기
answers = [data['answer'] for data in questions_keyword]
questions_keyword = list(chain(*answers))

In [4]:
questions_keyword[:4]

['전세 계약을 체결할 때 필요한 서류는 무엇인가요?',
 '전세 계약 시 주의해야 할 점은 무엇인가요?',
 '전세 계약 갱신 청구권은 어떻게 적용되나요?',
 '전세 계약을 해지하려면 어떤 절차를 밟아야 하나요?']

In [5]:
questions_cafe = jload("./data/questions_cafe_filtered.json")

In [6]:
questions_cafe[:4]

['신혼부부 집 선택 시 고려해야 할 사항은 무엇인가요?',
 '고소득 맞벌이 부부가 고민하는 매수 시 어떤 점을 고려해야 하나요?',
 '강남권 아파트 중에서 골프장이 잘 되어 있는 곳은 어디인가요?',
 '40평대 아파트의 입지 조건에 대해 알려주세요.']

In [7]:
questions = questions_cafe + questions_keyword

In [8]:
len(questions)

10243

selenium이 4.1 버전으로 업그레이드 되면서 별도로 webdriver를 다운로드 받을 필요가 없어졌습니다.  
webdriver 파일이 없으면 selenium이 자체적으로 다운로드하게 된다고 합니다. [stackoverflow](https://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-seleniumhttps://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-selenium)  
본 예제의 selenium 코드 크롤링 코드는 [wikidocs](https://wikidocs.net/137914) 내용을 참조하였습니다.  
아래 코드는 내부에 `multiprocessing`으로 구현되어 있으며, pool의 갯수는 CPU Core 갯수로 설정되어 있습니다.  
코드 실행시에 PC가 조금 느려질 수 있으니 참조해주세요.

In [10]:
from utils import get_document_through_selenium
from multiprocessing import cpu_count

search_data = get_document_through_selenium(
    crawling_type="search",
    inputs=questions,
    n_worker=cpu_count() - 2,
    n_documents=5,
    indent=4,
    mode="a",
    save_path = "./data/document.json"
)

10243it [1:34:02,  1.82it/s]


In [11]:
jload("./data/document.json")[0]

{'question': '신혼부부 집 선택 시 고려해야 할 사항은 무엇인가요?',
 'document': ['신혼부부이혼 시 고려해야 할 사항이 있다면 안녕하세요, 든든한 법률파트너 유진우변호사입니다. 결혼은 일생일대의 큰 사건이자 인생의 반환점이 될 수 있는 일이기 때문에 신중하게 생각하고 진행하는 경우가 많습니다. 내 배우자가 될 사람이 어떤 사람인지 살펴보고 이 사람과 함께 가정을 꾸렸을 때, 끝까지 함께할 수 있는지를 하게 되는데요. 이렇게 신중하게...',
  '신혼부부보험 가입하면서 간이식보험 정보 알아보세요 저는 원래 미갱신 건강보험만 준비했었는데, 최근 미갱신 건강보험 가입 추천 통화를 받았어요. 즉시 가입하는 것보다는 미갱신... 암 진단 시 보조인 비용도 중요하다는 것을 알고, 이런 부분들을 고려하여 가장 경제적이며 현명한 을 했다고 생각하였어요. 직접 상담보다는 자기 조사하고 탐색하는 것이 더...',
  '특히 신혼부부 침대는 고려할 사항이 너무 많다 보니 어떤 것을 봐야 할지 어려워하던 도중 좋은 매장을 만나 현명하게 침대 구매를 할 수 있었답니다. 씰리침대 아현점 1. 한국에서... 후회없는 선택이라고 자부해요. 순금 사은품까지 제공! 저는 매트리스와 프레임을 함께 구입해 300만원 이상 구매 증정받을 수 있는 순금까지 받았어요. 결혼 전 혼수를 받은 것...',
  '하며, 신혼부부 디딤돌대출의 한도는 최대 4억 원까지 입니다. 그리고 기간의 경우 만기 10, 15, 20, 30년으로 5년 단위로 책정이 되는데요. 거치는 최대 1년까지만 가능하니 이 점을 유의해 자산 계획을 세워보셔야 할 듯합니다. 그럼 금리는 어떻게 될까요? 해당 대출의 경우 2.45%에서 3.55% 사이로 금리가 책정됩니다만, 몇 가지 우대도 있어 이를 적용한다면 좀 더...',
  '가격, 면적, 시설 등을 고려하여 신혼부부의 라이프 스타일에 맞는 아파트를 선택해야 합니다. 4. 아파트 구매 시 유의해야 할 법적 사항 아파트 매매 시에는 법적인 사항도 유의해야 

가끔 크롤링이 안되는 경우가 있어서 한번 검수해줍니다.

In [4]:
search_data = jload('./data/document.json')
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

수집되지 않은 데이터를 한번더 크롤링하게 하고 `document.json`에 저장합니다.

In [15]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        search_data[idx] = get_document_through_selenium(
            crawling_type="search",
            inputs=search_data[idx]['question'],
            n_worker=cpu_count() - 2,
            n_documents=5,
            indent=4
        )[0]

1it [00:04,  4.16s/it]
1it [00:03,  3.48s/it]
1it [00:04,  4.02s/it]
1it [00:03,  3.22s/it]
1it [00:03,  3.93s/it]
1it [00:04,  4.09s/it]
1it [00:03,  3.93s/it]
1it [00:03,  3.65s/it]
1it [00:03,  3.43s/it]
1it [00:04,  4.11s/it]
1it [00:03,  3.76s/it]
1it [00:03,  3.28s/it]
1it [00:03,  3.77s/it]
1it [00:03,  3.20s/it]
1it [00:03,  3.63s/it]
1it [00:03,  3.47s/it]
1it [00:03,  3.08s/it]
1it [00:03,  3.46s/it]
1it [00:02,  2.96s/it]
1it [00:03,  3.66s/it]
1it [00:03,  3.24s/it]
1it [00:03,  3.23s/it]
1it [00:03,  3.83s/it]
1it [00:04,  4.04s/it]
1it [00:03,  3.33s/it]
1it [00:03,  3.32s/it]
1it [00:03,  3.70s/it]
1it [00:03,  3.36s/it]
1it [00:04,  4.25s/it]
1it [00:07,  7.26s/it]


제대로 수집되었는지 또 검사해줍니다.

In [5]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

크롤링이 왜 안됐는지 원인 파악을 위해 해당 질문으로 직접 검색을 해봅니다.  
검색해보면... "인기글" 항목이 검색되지 않습니다.  
따라서 해당 데이터는 삭제하였습니다.  

In [87]:
# 삭제할 인덱스 모으기
remove_idx_list = []
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        remove_idx_list.append(idx)

# 삭제할 인덱스를 pop 
for idx in remove_idx_list:
    search_data.pop(idx)

In [7]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

In [8]:
from utils import jsave

jsave(data=search_data, save_path="./data/document.json", mode="w", indent=4)

In [9]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

In [10]:
len(search_data)

9967

## <span style="color: #F2D388;"> 5. Instuction 데이터셋 만들기 </span>
완성된 질문리스트와 인기글 데이터를 통해 ChatGPT에 정답을 출력하도록 요청합니다.  
gpt API로 아래의 프롬프트처럼 question에 대한 answer 답변을 출력하도록 요청합니다.  
완성된 question, answer 텍스트는 학습시킬 모델의 훈련용 데이터셋으로 전달됩니다.  
본 문서의 프롬프트는 [Stanford Alpaca의 프롬프트](https://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txthttps://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txt)를 참조하였습니다.  

In [1]:
from prompts import INSTRUCTION_PROMPT_PREFIX, INSTRUCTION_PROMPT_CONTENT

print(INSTRUCTION_PROMPT_PREFIX + INSTRUCTION_PROMPT_CONTENT)

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러 개의 document가 주어지며, question은 각각 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. document의 정보가 question과 관련있는 정보라면 참조합니다, 관련이 없다면 참조하지 않습니다.
4. WebPilot을 이용하여 Google에서 검색을 하고, question과 관련있는 데이터를 참조하세요.
5. 위 3, 4번의 데이터를 바탕으로 답변하세요.
6. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어 정도로 구성해주세요.
9. 출력 형식은 JSON포맷을 따라야 하며, Indentation은 없도록 출력하세요.

출력 형식은 아래 포맷을 참조하세요:
{"question": "1주택자 종부세 폐지 여부가 확정된 건가요?", "answer": "현재 1주택자에 대한 종합부동산세 폐지...."}
{"question": "아파트 추이 분석에서 중요한 변수들은 무엇인가요?", "answer": "아파트 추이를 이해하기 위해 ...."}
###question:
{question}
###document:
{document}



In [2]:
from utils import jload

search = jload("./data/document.json")

In [3]:
# 리스트 형태인 document 데이터를 하나로 합치기
for data in search:
    data['document'] = "\n".join([f"{idx+1}. {d}" for idx, d in enumerate(data['document'])])

In [4]:
print(search[0]['document'])

1. 신혼부부이혼 시 고려해야 할 사항이 있다면 안녕하세요, 든든한 법률파트너 유진우변호사입니다. 결혼은 일생일대의 큰 사건이자 인생의 반환점이 될 수 있는 일이기 때문에 신중하게 생각하고 진행하는 경우가 많습니다. 내 배우자가 될 사람이 어떤 사람인지 살펴보고 이 사람과 함께 가정을 꾸렸을 때, 끝까지 함께할 수 있는지를 하게 되는데요. 이렇게 신중하게...
2. 신혼부부보험 가입하면서 간이식보험 정보 알아보세요 저는 원래 미갱신 건강보험만 준비했었는데, 최근 미갱신 건강보험 가입 추천 통화를 받았어요. 즉시 가입하는 것보다는 미갱신... 암 진단 시 보조인 비용도 중요하다는 것을 알고, 이런 부분들을 고려하여 가장 경제적이며 현명한 을 했다고 생각하였어요. 직접 상담보다는 자기 조사하고 탐색하는 것이 더...
3. 특히 신혼부부 침대는 고려할 사항이 너무 많다 보니 어떤 것을 봐야 할지 어려워하던 도중 좋은 매장을 만나 현명하게 침대 구매를 할 수 있었답니다. 씰리침대 아현점 1. 한국에서... 후회없는 선택이라고 자부해요. 순금 사은품까지 제공! 저는 매트리스와 프레임을 함께 구입해 300만원 이상 구매 증정받을 수 있는 순금까지 받았어요. 결혼 전 혼수를 받은 것...
4. 하며, 신혼부부 디딤돌대출의 한도는 최대 4억 원까지 입니다. 그리고 기간의 경우 만기 10, 15, 20, 30년으로 5년 단위로 책정이 되는데요. 거치는 최대 1년까지만 가능하니 이 점을 유의해 자산 계획을 세워보셔야 할 듯합니다. 그럼 금리는 어떻게 될까요? 해당 대출의 경우 2.45%에서 3.55% 사이로 금리가 책정됩니다만, 몇 가지 우대도 있어 이를 적용한다면 좀 더...
5. 가격, 면적, 시설 등을 고려하여 신혼부부의 라이프 스타일에 맞는 아파트를 선택해야 합니다. 4. 아파트 구매 시 유의해야 할 법적 사항 아파트 매매 시에는 법적인 사항도 유의해야 합니다. 계약서 작성 및 검토, 중개사와의 협의, 유지 보수 지원 등을 포함하여 법적인 절차를 정확히 이행해야 합니다.

In [5]:
print(INSTRUCTION_PROMPT_CONTENT)


###question:
{question}
###document:
{document}



In [6]:
prefix = INSTRUCTION_PROMPT_PREFIX
instructions = []
total_instructions = len(search) // 10
for index in range(1, total_instructions + 2):
    start_index = (index - 1) * 10
    end_index = index * 10
    content = "\n".join([
        INSTRUCTION_PROMPT_CONTENT.format(question = data['question'], document = data['document'])
        for data in search[start_index:end_index]
    ])
    instruction = prefix + content
    instructions.append(instruction)

In [7]:
print(instructions[0][:1000])

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러 개의 document가 주어지며, question은 각각 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. document의 정보가 question과 관련있는 정보라면 참조합니다, 관련이 없다면 참조하지 않습니다.
4. WebPilot을 이용하여 Google에서 검색을 하고, question과 관련있는 데이터를 참조하세요.
5. 위 3, 4번의 데이터를 바탕으로 답변하세요.
6. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어 정도로 구성해주세요.
9. 출력 형식은 JSON포맷을 따라야 하며, Indentation은 없도록 출력하세요.

출력 형식은 아래 포맷을 참조하세요:
{"question": "1주택자 종부세 폐지 여부가 확정된 건가요?", "answer": "현재 1주택자에 대한 종합부동산세 폐지...."}
{"question": "아파트 추이 분석에서 중요한 변수들은 무엇인가요?", "answer": "아파트 추이를 이해하기 위해 ...."}
###question:
신혼부부 집 선택 시 고려해야 할 사항은 무엇인가요?
###document:
1. 신혼부부이혼 시 고려해야 할 사항이 있다면 안녕하세요, 든든한 법률파트너 유진우변호사입니다. 결혼은 일생일대의 큰 사건이자 인생의 반환점이 될 수 있는 일이기 때문에 신중하게 생각하고 진행하는 경우가 많습니다. 내 배우자가 될 사람이 어떤 사람인지 살펴보고 이 사람과 함께 가정을 꾸렸을 때, 끝까지 함께할 수 있는지를 하게 되는데요. 이렇게 신중하게...
2. 신혼부부보험 가입하면서 간이식보험 정보 알아보세요 저는 원래 미갱신 건강보험만 준비했었는데, 최근 미갱신 건강보험 가입 추천 통화를 받았어요. 즉시 가입하는 것보다는 미갱신... 암

종종 중간에 유니코드 에러가 발생해서 전체 작업이 멈추는 경우가 있었습니다.  
중간에 에러가 발생해도 Skip하도록하고, 에러는 별도로 error_prompts 리스트에 추가하도록 합니다.  
API로 응답받은 데이터는 파일로 저장하도록 합니다.  
병렬처리시 파일 I/O의 race condition을 방지하기 위해 Lock으로 Thread safe하게 관리합니다.

In [13]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from tqdm import tqdm
from utils import get_completion
from threading import Lock
import os

error_prompts = []

# https://superfastpython.com/thread-mutex-lock/
# completion에 유니코드 에러가 자주 발생해서 병렬처리시 저장할 수 있도록 개선 / 예외처리 추가
def _thread_safe_get_completion(prompt, model = "gpt-4o"):
    try:
        result = get_completion(model=model, prompt=prompt)
    except Exception as e:
        question = prompt.split("###question:\n")[1].split("###document")[0][:-1]
        print(f"오류 발생한 questions: {question}")
        error_prompts.append(prompt)
        print(e)
        return None
    lock = Lock() # I/O에서 Race Condition이 발생하지 않도록
    with lock:
        with open("./data/instruction.jsonl", "a", encoding="utf-8") as f:
            for line in result.split("\n"):
                f.write(line)
                f.write("\n")
            return result
    

with ThreadPoolExecutor() as pool:
    results = list(tqdm(pool.map(partial(_thread_safe_get_completion, model="gpt-4o"), instructions)))

997it [37:38,  2.27s/it]


`error_prompts` 내에 에러가 몇개 있었으나 많지 않아서 해당 데이터는 그냥 제외했습니다.  
줄바꿈이 2번있는 경우 `jsonl`로 읽기 어려워서 후처리를 해줍니다.  
[LangChain](https://github.com/langchain-ai/langchain)이나 [openai-python](https://github.com/openai/openai-python)를 이용해서 더 강력하게 프롬프트를 구성하면, 이러한 후처리 작업이 없을 것 같습니다.

In [19]:
with open("./data/instruction.jsonl", "r") as f:
    parsed = f.readlines()
parsed = [lines.replace("\n", "") for lines in parsed if lines != "\n"]

In [20]:
len(parsed)

9791

In [22]:
from utils import jsave

jsave(data=parsed, save_path="./data/instruction.jsonl", mode="w")

In [23]:
from utils import jload

len(jload("./data/instruction.jsonl"))

9791

## <span style="color: #F2D388;"> Review </span>

학습용 데이터셋을 만들었습니다.  
학습용 데이터셋을 만드는 방법은 다양하고 도메인에 맞게 적용할 수 있습니다.  
LLM을 활용해 데이터셋을 만드는 방법은 Self-Instruct([https://arxiv.org/pdf/2212.10560](https://arxiv.org/pdf/2212.10560))을 참고하면 좋을 것 같습니다.

논문에서는 다양한 Instruction을 생성하기 위해 ROUGE-L 스코어를 활용하기도 합니다.  
모델이 생성한 Instruction중 ROUGE-L 스코어가 0.7보다 낮은 Instruction을 다음 Instruction 생성때 활용하기도 합니다.

이 예제에서는 하나의 키워드에 여러 Instruction을 생성하는 등 중복된 Instruction이 있을 수 있습니다.  
하지만 이 예제는 모델 사이즈도 작고, 예산상 만들 수 있는 데이터셋도 한정되어있기 때문에 어느정도 오버피팅을 위해서 유사한 Instruction을 필터링하지 않았습니다.
실제로 모델을 개발한다면 이러한 부분도 고려하여 개발되어야 합니다.

또한 GPT의 경우 `gpt-3.5-turbo`를 사용하려면 프롬프트를 좀더 강력하게 구축해야 했습니다.(많은 수작업이 소요됩니다.)  
다행히도 `gpt-4o`가 출시된 덕에 저렴한 가격으로 데이터셋을 구축할 수 있었습니다.  

질좋은 답변을 위해 아래 2가지를 잘 고려해야합니다.
+ 사람들이 많이 연구한 양질의 프롬프트를 사용하기
+ Few-shot 추가하기  

## <span style="color: #F2D388;"> References </span>
+ [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txt)
+ [awesome-chatgpt-prompts](https://github.com/f/awesome-chatgpt-prompts)
+ [Self-Instruct: Aligning Language Models with Self-Generated Instructions](https://arxiv.org/pdf/2212.10560)
+ [Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena](https://arxiv.org/pdf/2306.05685)
+ [chatgpt-prompt-engineering-for-developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers)
+ [openai-python](https://github.com/openai/openai-python)